In [1]:
from ultralytics import YOLO
import numpy as np
import torch
import torch.nn as nn
import torchvision

In [2]:
# 1. Загрузка предобученной YOLOv10 модели
model = YOLO('yolov9e.pt')  # Замените на фактический путь к модели YOLOv10

# 2. Фильтрация на основе класса "человек" (class_id = 0 для COCO)
def filter_persons(results):
    filtered_results = []
    for result in results:
        person_boxes = [box for box in result.boxes if int(box.cls) == 0]  # Класс "person" в COCO
        result.boxes = person_boxes
        filtered_results.append(result)
    return filtered_results


In [4]:
# 3. Использование модели для детекции на изображении
image_path = 'test/images/5.jpg'
results = model(image_path)

# Фильтрация результатов только по классу "человек"
results = filter_persons(results)

# Визуализация детекций
results[0].plot(save=False, conf=True)  # Сохраняет изображение с детекциями


image 1/1 C:\Users\user\PycharmProjects\spbu_deep_learning1\homeworks\rnn_hw\test\images\5.jpg: 480x640 12 persons, 1 motorcycle, 181.8ms
Speed: 0.0ms preprocess, 181.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


array([[[13, 35, 46],
        [15, 37, 48],
        [16, 38, 49],
        ...,
        [28, 54, 54],
        [28, 54, 54],
        [30, 56, 56]],

       [[15, 37, 48],
        [15, 37, 48],
        [15, 37, 48],
        ...,
        [26, 55, 52],
        [27, 55, 55],
        [28, 57, 54]],

       [[13, 37, 49],
        [13, 37, 49],
        [13, 37, 49],
        ...,
        [26, 58, 53],
        [27, 58, 55],
        [28, 60, 55]],

       ...,

       [[42, 37, 38],
        [53, 48, 49],
        [57, 52, 53],
        ...,
        [14, 18, 23],
        [24, 30, 35],
        [27, 33, 40]],

       [[42, 37, 38],
        [45, 40, 41],
        [49, 44, 45],
        ...,
        [17, 21, 26],
        [18, 24, 31],
        [18, 24, 31]],

       [[36, 31, 32],
        [42, 37, 38],
        [44, 39, 40],
        ...,
        [22, 26, 31],
        [14, 20, 27],
        [13, 21, 28]]], dtype=uint8)

In [3]:
def detect_people(image_path):
    """
    Выполняет детекцию людей на изображении и возвращает ограничивающие рамки.
    """
    results = model(image_path)  # Выполняем предсказание
    boxes = []
    for box in results[0].boxes:
        if int(box.cls) == 0:  # Класс "person"
            left, top, right, bottom = box.xyxy[0].tolist()
            boxes.append((int(left), int(top), int(right), int(bottom)))
    return boxes

In [8]:
import pandas as pd
import os

# Сбор всех изображений из тестового набора
image_files = [f for f in os.listdir('test/images/') if f.endswith(('.jpg', '.png'))]
image_files.sort()  # Убедитесь, что сортировка соответствует ID

# Создание файла submission.csv
submission_data = []
for i, image_file in enumerate(image_files):
    image_path = os.path.join('test/images/', image_file)
    bboxes = detect_people(image_path)
    submission_data.append({"ID": image_file[:-4], "bboxes": bboxes})

# Создание DataFrame и запись в CSV
df = pd.DataFrame(submission_data)
df["bboxes"] = df["bboxes"].apply(lambda x: str(x))  # Преобразуем список в строку
df.to_csv("submission1.csv", index=False)
print(f"Submission file saved to submission1.csv")


image 1/1 C:\Users\user\PycharmProjects\spbu_deep_learning1\homeworks\rnn_hw\test\images\100.jpg: 480x640 10 persons, 1 bird, 251.1ms
Speed: 0.0ms preprocess, 251.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\user\PycharmProjects\spbu_deep_learning1\homeworks\rnn_hw\test\images\1000.jpg: 640x512 (no detections), 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 C:\Users\user\PycharmProjects\spbu_deep_learning1\homeworks\rnn_hw\test\images\1006.jpg: 640x640 (no detections), 31.3ms
Speed: 0.0ms preprocess, 31.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 C:\Users\user\PycharmProjects\spbu_deep_learning1\homeworks\rnn_hw\test\images\1007.jpg: 640x512 1 person, 2 couchs, 28.6ms
Speed: 0.0ms preprocess, 28.6ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 C:\Users\user\PycharmProjects\spbu_deep_learning1\homeworks\rnn_hw

PermissionError: [Errno 13] Permission denied: 'submission1.csv'

In [10]:
df.to_csv("submission2.csv", index=False)
print(f"Submission file saved to submission1.csv")

Submission file saved to submission1.csv


In [5]:
import os
from PIL import Image

# Параметры
images_path = "train/images_all"  # Папка с изображениями
labels_path = "train/labels_all"  # Папка с аннотациями
output_images_path = "train/images"  # Папка для сохранения изображений
output_labels_path = "train/labels"  # Папка для сохранения аннотаций

# Убедимся, что выходные папки существуют
os.makedirs(output_images_path, exist_ok=True)
os.makedirs(output_labels_path, exist_ok=True)

# Целевой размер изображений
target_size = (640, 640)

def process_annotation(label_path, original_size, new_size):
    """
    Конвертирует аннотации из абсолютных координат в относительные (YOLO формат).
    """
    old_width, old_height = original_size
    new_width, new_height = new_size
    new_annotations = []

    with open(label_path, "r") as file:
        for line in file:
            parts = line.strip().split(',')
            x_min, y_min, x_max, y_max = map(float, parts)

            # Перевод координат в новый масштаб
            x_min = x_min / old_width
            y_min = y_min / old_height
            x_max = x_max / old_width
            y_max = y_max / old_height

            # Вычисление координат центра и размеров в YOLO формате
            x_center = (x_min + x_max) / 2
            y_center = (y_min + y_max) / 2
            width = x_max - x_min
            height = y_max - y_min

            # Формируем новую строку
            new_annotations.append(f"{int(0)} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

    return new_annotations

# Проходим по всем изображениям и аннотациям
for image_file in os.listdir(images_path):
    if image_file.endswith(('.jpg', '.png', '.jpeg')):
        # Пути к изображению и аннотации
        image_path = os.path.join(images_path, image_file)
        label_path = os.path.join(labels_path, os.path.splitext(image_file)[0] + ".txt")
        output_image_path = os.path.join(output_images_path, image_file)
        output_label_path = os.path.join(output_labels_path, os.path.splitext(image_file)[0] + ".txt")

        # Открытие изображения и его ресайз
        with Image.open(image_path) as img:
            original_size = img.size
            img_resized = img.resize(target_size)
            img_resized.save(output_image_path)

        # Обработка аннотации
        if os.path.exists(label_path):
            new_annotations = process_annotation(label_path, original_size, target_size)
            with open(output_label_path, "w") as output_file:
                output_file.write("\n".join(new_annotations))

print("Все изображения и аннотации успешно обработаны.")


Все изображения и аннотации успешно обработаны.


In [4]:
# Fine-tuning
model.train(data='data.yaml', epochs=100, imgsz=640, batch=0.9)

# 5. Сохранение дообученной модели
model.save('yolov9e_person_finetuned.pt')

Ultralytics 8.3.40  Python-3.10.11 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
engine\trainer: task=detect, mode=train, model=yolov9e.pt, data=data.yaml, epochs=100, time=None, patience=100, batch=0.9, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train142, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning C:\Users\user\PycharmProjects\spbu_deep_learning1\homeworks\rnn_hw\train\labels.cache... 3321 images, 2526 backgrounds, 0 corrupt: 100%|██████████| 3321/3321 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 90.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER) 15.99G total, 5.53G reserved, 1.58G allocated, 8.88G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    58145683       192.7         1.892          62.9         46.87        (1, 3, 640, 640)                    list
    58145683       385.4         3.160         78.12          62.5        (2, 3, 640, 640)                    list
    58145683       770.7         5.390         109.4          62.5        (4, 3, 640, 640)                    list
    58145683        1541        10.066         156.2         93.75        (8, 3, 640, 640)                    list
    58145683        3083        19.390          1454         969.6       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 6 for CUDA:0 14.86G/15.99G (93%) 


train: Scanning C:\Users\user\PycharmProjects\spbu_deep_learning1\homeworks\rnn_hw\train\labels.cache... 3321 images, 2526 backgrounds, 0 corrupt: 100%|██████████| 3321/3321 [00:00<?, ?it/s]

...

    100/100      8.81G        nan        nan        nan          5        640: 100%|██████████| 554/554 [01:56<00:00,  4.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  7.89it/s]

                   all         23         10    0.00176        0.9    0.00846    0.00428



100 epochs completed in 3.315 hours.
Optimizer stripped from C:\Users\user\PycharmProjects\spbu_deep_learning1\runs\detect\train142\weights\last.pt, 117.3MB
Optimizer stripped from C:\Users\user\PycharmProjects\spbu_deep_learning1\runs\detect\train142\weights\best.pt, 117.3MB

Validating C:\Users\user\PycharmProjects\spbu_deep_learning1\runs\detect\train142\weights\best.pt...
Ultralytics 8.3.40  Python-3.10.11 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLOv9e summary (fused): 687 layers, 57,377,171 parameters, 0 gradients, 189.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.33it/s]


                   all         23         10    0.00158        0.9     0.0215    0.00375
Speed: 0.7ms preprocess, 11.7ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to C:\Users\user\PycharmProjects\spbu_deep_learning1\runs\detect\train142
